# Домашнее задание № 3. Исправление опечаток

## 1. Доп. ранжирование по вероятности (3 балла)

Дополните get_closest_hybrid_match в семинаре так, чтобы из кандадатов с одинаковым расстоянием редактирования выбиралось наиболее вероятное.

In [2]:
import os, re
from string import punctuation
import numpy as np
import pandas as pd

from collections import Counter

import textdistance
from difflib import get_close_matches

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity, cosine_distances
from sklearn.metrics import classification_report, accuracy_score

from tqdm.notebook import tqdm

In [3]:
corpus = open('/Users/alexandradolidze/Desktop/compling/data/wiki_data.txt', encoding='utf8').read()
vocab = Counter(re.findall('\w+', corpus.lower()))
N = sum(vocab.values())

In [4]:
bad = open('/Users/alexandradolidze/Desktop/compling/data/sents_with_mistakes.txt', encoding='utf8').read().splitlines()
true = open('/Users/alexandradolidze/Desktop/compling/data/correct_sents.txt', encoding='utf8').read().splitlines()

In [5]:
def P(word, N = N):
    return vocab[word] / N

def predict_mistaken(word, vocab):
    return 0 if word in vocab else 1
def get_closest_match_with_metric(text, lookup,topn=20, metric=textdistance.levenshtein):
    similarities = Counter()
    
    for word in lookup:
        similarities[word] = metric.normalized_similarity(text, word) 
    
    return similarities.most_common(topn)

def get_closest_match_vec(text, X, vec, topn=20):
    v = vec.transform([text])
    similarities = cosine_distances(v, X)[0]
    topn = similarities.argsort()[:topn] 
    
    return [(id2word[top], similarities[top]) for top in topn]

def get_closest_hybrid_match(text, X, vec, topn=3, metric=textdistance.damerau_levenshtein):
    candidates = get_closest_match_vec(text, X, vec, topn*4)
    lookup = [cand[0] for cand in candidates]
    closest = get_closest_match_with_metric(text, lookup, topn, metric=metric)
    
#     return closest
    distance = []
    for i in closest:
        item = list(i)
        item.append(P(i[0]))
        distance.append(item)
        
    # Кирилл, спасибо за совет!
    dist_to_prob = pd.DataFrame(distance, columns = ['word', 'distance', 'probability'])
    dist_to_prob = dist_to_prob.sort_values(by = ['distance', 'probability'], ascending = [False, False]).reset_index(drop=True)

    probabliest = []
    
    print(dist_to_prob)
    
    unique_dist = list(dist_to_prob.distance.unique())
    for i in range(len(dist_to_prob)):
        if dist_to_prob.distance[i] in unique_dist:
            print(dist_to_prob.word[i], dist_to_prob.distance[i])
            probabliest.append(dist_to_prob.word[i])
            probabliest.append(dist_to_prob.distance[i])
            unique_dist.remove(dist_to_prob.distance[i])
            
    return probabliest


In [6]:
word2id = list(vocab.keys())
id2word = {i:word for i, word in enumerate(vocab)}


vec = CountVectorizer(analyzer='char', ngram_range=(1,1))
X = vec.fit_transform(vocab)

In [7]:
def align_words(sent_1, sent_2):
    tokens_1 = sent_1.lower().split()
    tokens_2 = sent_2.lower().split()
    
    tokens_1 = [token.strip(punctuation) for token in tokens_1]
    tokens_2 = [token.strip(punctuation) for token in tokens_2]
    
    tokens_1 = [token for token in tokens_1 if token]
    tokens_2 = [token for token in tokens_2 if token]
    
    assert len(tokens_1) == len(tokens_2)
    
    return list(zip(tokens_1, tokens_2))

cashed = {}

mistakes = []
total_mistaken = 0
mistaken_fixed = 0

total_correct = 0
correct_broken = 0

total = 0
correct = 0

for i in tqdm(range(len(true))):
    word_pairs = align_words(true[i], bad[i])
    for pair in word_pairs:
        if predict_mistaken(pair[1], vocab):
            pred = cashed.get(pair[1], get_closest_hybrid_match(pair[1], X, vec)[0])
            cashed[pair[1]] = pred
        else:
            pred = pair[1]
        
            
        if pred == pair[0]:
            correct += 1
        else:
            mistakes.append((pair[0], pair[1], pred))
        total += 1
            
        if pair[0] == pair[1]:
            total_correct += 1
            if pair[0] != pred:
                correct_broken += 1
        else:
            total_mistaken += 1
            if pair[0] == pred:
                mistaken_fixed += 1

  0%|          | 0/915 [00:00<?, ?it/s]

          word  distance   probability
0  симпатичнее  0.785714  1.939759e-07
1   симпатичен  0.642857  1.939759e-07
2   опаснейшим  0.428571  1.939759e-07
симпатичнее 0.7857142857142857
симпатичен 0.6428571428571428
опаснейшим 0.4285714285714286
        word  distance   probability
0  шпионской  0.888889  9.698796e-07
1   шпионско  0.888889  1.939759e-07
2  шпионском  0.888889  1.939759e-07
шпионской 0.8888888888888888
         word  distance   probability
0   гламурным  0.888889  3.879518e-07
1   гламурные  0.888889  1.939759e-07
2  гламурными  0.800000  1.939759e-07
гламурным 0.8888888888888888
гламурными 0.8
            word  distance   probability
0         spread  0.600000  1.939759e-07
1  superspeedway  0.538462  3.879518e-07
2         shared  0.400000  1.939759e-07
spread 0.6
superspeedway 0.5384615384615384
shared 0.4
    word  distance   probability
0  clamp       0.8  1.939759e-07
1  clasp       0.8  1.939759e-07
2    lac       0.5  1.939759e-07
clamp 0.8
lac 0.5
         wo

            word  distance  probability
0    участвовать  0.916667     0.000033
1    чувствовать  0.833333     0.000002
2  почувствовать  0.769231     0.000003
участвовать 0.9166666666666666
чувствовать 0.8333333333333334
почувствовать 0.7692307692307692
       word  distance   probability
0    сиднея  0.666667  2.715663e-06
1  соседняя  0.625000  1.939759e-06
2  середняя  0.625000  1.939759e-07
сиднея 0.6666666666666667
соседняя 0.625
            word  distance   probability
0       питаться  0.727273  3.491567e-06
1  притягиваться  0.692308  1.939759e-07
2    причитаться  0.636364  1.939759e-07
питаться 0.7272727272727273
притягиваться 0.6923076923076923
причитаться 0.6363636363636364
         word  distance   probability
0  опухолевых       0.5  1.745783e-06
1       лопух       0.5  5.819278e-07
2    поцелуях       0.5  1.939759e-07
опухолевых 0.5
               word  distance  probability
0   ответственность  0.933333     0.000027
1  ответственностью  0.875000     0.000003
2   отве

     word  distance   probability
0  казака  0.666667  4.655422e-06
1  капака  0.666667  3.879518e-07
2  каахка  0.666667  1.939759e-07
казака 0.6666666666666667
         word  distance   probability
0     хочется  0.875000  6.401205e-06
1  сочетаться  0.700000  1.939759e-06
2   захочется  0.666667  1.939759e-07
хочется 0.875
сочетаться 0.7
захочется 0.6666666666666667
        word  distance   probability
0   взводить  0.875000  3.879518e-07
1  возводить  0.777778  1.163856e-06
2     водить  0.750000  5.819278e-07
взводить 0.875
возводить 0.7777777777777778
водить 0.75
     word  distance   probability
0  пимена  0.857143  1.357831e-06
1  симена  0.714286  1.939759e-07
2  синема  0.428571  1.163856e-06
пимена 0.8571428571428572
симена 0.7142857142857143
синема 0.4285714285714286
           word  distance   probability
0  супермаркете  0.923077  9.698796e-07
1   супермаркет  0.846154  2.133735e-06
2  супермаркеты  0.846154  3.879518e-07
супермаркете 0.9230769230769231
супермаркет 0.8461

      word  distance   probability
0  wapalta  0.571429  1.745783e-06
1     pata  0.500000  3.879518e-07
2   ataxia  0.500000  1.939759e-07
wapalta 0.5714285714285714
pata 0.5
       word  distance   probability
0  опасался     0.875  4.073494e-06
1  спасался     0.750  3.879518e-07
2  опасаясь     0.625  1.183253e-05
опасался 0.875
спасался 0.75
опасаясь 0.625
          word  distance   probability
0       сжился  0.750000  1.939759e-07
1   рассорился  0.700000  5.819278e-07
2  расстроился  0.636364  1.939759e-07
сжился 0.75
рассорился 0.7
расстроился 0.6363636363636364
    word  distance   probability
0  сабба       0.8  7.759037e-07
1   саба       0.8  5.819278e-07
2   сажа       0.8  1.939759e-07
сабба 0.8
      word  distance   probability
0  однажды     0.875  4.442049e-05
1   джандо     0.375  1.939759e-07
2    джоны     0.375  1.939759e-07
однажды 0.875
джандо 0.375
          word  distance   probability
0  обезумевший  0.818182  3.879518e-07
1  обезумевшей  0.818182  1.939759e

...

        word  distance   probability
0  lorillard       0.6  7.759037e-07
1   collider       0.5  3.879518e-07
2    coralie       0.5  3.879518e-07
lorillard 0.6
collider 0.5
        word  distance   probability
0      blanc  0.833333  1.939759e-07
1    balance  0.714286  3.879518e-07
2  balkanica  0.666667  1.939759e-07
blanc 0.8333333333333334
balance 0.7142857142857143
balkanica 0.6666666666666667
       word  distance   probability
0     пеште  0.833333  1.551807e-06
1  помешает  0.625000  3.879518e-07
2  потешное  0.625000  1.939759e-07
пеште 0.8333333333333334
помешает 0.625
       word  distance   probability
0    нищета  0.666667  1.163856e-06
1  навещает  0.625000  1.163856e-06
2   затащен  0.428571  1.939759e-07
нищета 0.6666666666666667
навещает 0.625
затащен 0.4285714285714286
           word  distance   probability
0  складываются  0.916667  2.715663e-06
1  укладываются  0.916667  3.879518e-07
2  вкладываются  0.916667  1.939759e-07
складываются 0.9166666666666666
        

       word  distance   probability
0  ответьте     0.875  1.939759e-07
1    ответь     0.750  5.819278e-07
2    ответе     0.750  1.939759e-07
ответьте 0.875
ответь 0.75
         word  distance   probability
0      адресу  0.857143  1.532410e-05
1  адресуются  0.700000  1.939759e-07
2     андерсу  0.571429  1.939759e-07
адресу 0.8571428571428572
адресуются 0.7
андерсу 0.5714285714285714
        word  distance   probability
0   барракуд  0.888889  1.939759e-07
1  баррикаду  0.777778  1.939759e-07
2      браку  0.555556  4.267470e-06
барракуд 0.8888888888888888
баррикаду 0.7777777777777778
браку 0.5555555555555556
            word  distance  probability
0   периодически  0.916667     0.000023
1  периодических  0.846154     0.000007
2  периодические  0.846154     0.000004
периодически 0.9166666666666666
периодических 0.8461538461538461
                  word  distance   probability
0        созванивается  0.923077  1.939759e-07
1  восстанавливающемся  0.526316  1.939759e-07
2            

          word  distance   probability
0  постарались  0.636364  9.698796e-07
1      тобиаса  0.500000  3.879518e-07
2     отписала  0.500000  1.939759e-07
постарались 0.6363636363636364
тобиаса 0.5
       word  distance   probability
0  простому  0.750000  1.163856e-06
1    постом  0.714286  3.103615e-06
2    постум  0.571429  7.759037e-07
простому 0.75
постом 0.7142857142857143
постум 0.5714285714285714
               word  distance   probability
0  сногсшибательной  0.875000  1.939759e-07
1        сигнальной  0.571429  3.879518e-07
2         насильной  0.571429  1.939759e-07
сногсшибательной 0.875
сигнальной 0.5714285714285714
       word  distance  probability
0   удается     0.875     0.000009
1  делается     0.750     0.000011
2    дается     0.750     0.000003
удается 0.875
делается 0.75
           word  distance   probability
0     благодаря  0.777778  2.368446e-04
1    благодарят  0.700000  9.698796e-07
2  поблагодарил  0.500000  2.327711e-06
благодаря 0.7777777777777778
благо

       word  distance   probability
0      воот  0.666667  1.939759e-07
1  оборотов  0.375000  4.073494e-06
2  ротового  0.375000  7.759037e-07
воот 0.6666666666666667
оборотов 0.375
            word  distance   probability
0     позитивной    0.6250  3.879518e-07
1      позитивно    0.5625  1.745783e-06
2  оппозиционной    0.4375  2.327711e-06
позитивной 0.625
позитивно 0.5625
оппозиционной 0.4375
         word  distance   probability
0     хочется  0.875000  6.401205e-06
1  сочетаться  0.700000  1.939759e-06
2   захочется  0.666667  1.939759e-07
хочется 0.875
сочетаться 0.7
захочется 0.6666666666666667
       word  distance   probability
0  съездила     0.875  3.879518e-07
1    съезда     0.750  2.230723e-05
2    съезде     0.750  2.230723e-05
съездила 0.875
съезда 0.75
        word  distance  probability
0   пообещал  0.727273     0.000007
1  пообещали  0.636364     0.000002
2  пообещала  0.636364     0.000001
пообещал 0.7272727272727273
пообещали 0.6363636363636364
      word  dist

      word  distance   probability
0  цинично  0.571429  1.939759e-07
1     ночи  0.500000  2.056145e-05
2     чино  0.500000  9.698796e-07
цинично 0.5714285714285714
ночи 0.5
      word  distance   probability
0   валдас       0.5  3.879518e-07
1   освала       0.5  1.939759e-07
2  салвадо       0.4  1.939759e-07
валдас 0.5
салвадо 0.4
        word  distance   probability
0  мудрейший  0.888889  1.939759e-07
1    умерший  0.625000  2.133735e-06
2    умершие  0.500000  1.551807e-06
мудрейший 0.8888888888888888
умерший 0.625
умершие 0.5
         word  distance   probability
0   некоторые  0.888889  2.777735e-04
1  героторные  0.700000  3.879518e-07
2      ноорте  0.625000  3.879518e-07
некоторые 0.8888888888888888
героторные 0.7
ноорте 0.625
     word  distance   probability
0   пусть       0.8  2.385904e-05
1   пульс       0.6  9.698796e-07
2  пульса       0.5  9.698796e-07
пусть 0.8
пульс 0.6
пульса 0.5
      word  distance   probability
0  несутся     0.625  1.939759e-07
1    сетуя  

In [8]:
print(correct/total * 100)
print(mistaken_fixed/total_mistaken * 100)
print(correct_broken/total_correct * 100)

84.77238619309655
42.701863354037265
9.004249454461927


In [9]:
# Результаты семинара

# 84.7023511755878
# 42.1583850931677
# 9.004249454461927

## 2.  Symspell (5 баллов)

Реализуйте алгоритм Symspell. Он похож на алгоритм Норвига, но проще и быстрее. Там к словам применяется только одна операция - удаление символа. Описание алгоритма по шагам:

1) Составляется словарь правильных слов  
2) На основе словаря правильных слов составляется словарь удалений - для каждого правильного слова создаются все варианты удалений и создается словарь, где ключ - слово с удалением, а значение - правильное слово   
3) Для выбора исправления для слова с опечаткой генерируются все варианты удаления, из них выбираются те, что есть в словаре удалений, построенного на шаге 2. Слово с опечаткой заменяется на правильное слово, соответствующее варианту удаления  
4) Если в словаре удалений есть несколько вариантов, то выбирается удаление, которому соответствует наиболее вероятное правильное слово  


Оцените качество полученного алгоритма теми же тремя метриками.

In [11]:
def generate_errors(word):    
    splits = [(word[:i], word[i:]) for i in range(len(word) + 1)]
    return [L + R[1:] for L, R in splits if R]

wrong_right = {}
for v in tqdm(vocab):
    for w in generate_errors(v):
        if w and not w.isdigit():
            if not w in wrong_right:
                wrong_right[w] = [v]
            else:
                wrong_right[w].append(v)

  0%|          | 0/368802 [00:00<?, ?it/s]

In [12]:
def P(word, N = N):
    return vocab[word] / N

In [13]:
def symspell(vocab, word):
    w_errors = generate_errors(word)
    d_candidates = []
    for i in w_errors:
        if i in wrong_right:
            d_candidates.extend(wrong_right.get(i))
        elif i in vocab:
            d_candidates.append(i)
    if d_candidates:
        return max(d_candidates, key=P)
    return word

In [14]:
def align_words(sent_1, sent_2):
    tokens_1 = sent_1.lower().split()
    tokens_2 = sent_2.lower().split()
    
    tokens_1 = [token.strip(punctuation) for token in tokens_1]
    tokens_2 = [token.strip(punctuation) for token in tokens_2]
    
    tokens_1 = [token for token in tokens_1 if token]
    tokens_2 = [token for token in tokens_2 if token]
    
    assert len(tokens_1) == len(tokens_2)
    
    return list(zip(tokens_1, tokens_2))

cashed = {}

mistakes = []
total_mistaken = 0
mistaken_fixed = 0

total_correct = 0
correct_broken = 0

total = 0
correct = 0

for i in tqdm(range(len(true))):
    word_pairs = align_words(true[i], bad[i])
    for pair in word_pairs:
        if predict_mistaken(pair[1], vocab):
            pred = cashed.get(pair[1], symspell(vocab, pair[1]))
            cashed[pair[1]] = pred
        else:
            pred = pair[1]
        
            
        if pred == pair[0]:
            correct += 1
        else:
            mistakes.append((pair[0], pair[1], pred))
        total += 1
            
        if pair[0] == pair[1]:
            total_correct += 1
            if pair[0] != pred:
                correct_broken += 1
        else:
            total_mistaken += 1
            if pair[0] == pred:
                mistaken_fixed += 1

  0%|          | 0/915 [00:00<?, ?it/s]

In [15]:
print(correct/total * 100)
print(mistaken_fixed/total_mistaken * 100)
print(correct_broken/total_correct * 100)

86.22311155577789
27.018633540372672
5.0189502698977835


## *3. Чтение. (2 балла)

Прочитайте эту главу в книге Speech and Language Processing - https://web.stanford.edu/~jurafsky/slp3/2.pdf .
Ответьте на следующий вопрос:

1. Что такое Byte-Pair Encoding (опишите по-русски, как минимум 10 предложениями)?

*Это задание не связано напрямую с исправлением опечаток, но это важная тема, к которой мы вернемся в конце курса

Byte-pair encoding или BPE - простейший из семейства алгоритмов токенизации, которые рассматривают токен не как слово или символ, а как сочетание подстрок, которые в свою очередь также считаются токенами. Такой подход к понимаю токена продуктивен в работе с несловарными, незнакомыми единицами (нецензурной лексикой, неологизмами, окказионализми).

Непосредственно BPE алгоритм выполняется обычно пословно, поэтому изначально весь словарь состоит из отдельных единичных символов (букв, например), отдельным символом считается и конец слова. Алгоритм проходит по корпусу и считает частотность всех стоящих рядом символов. Самая часто встречающаяся последовательность символов добавляется в словарь как один символ, после чего алгоритм заменяются все такие пары символов в корпусе одним, добавляя этот обобщенный символ в словарь, после чего алгоритм снова ищет самую частотную последовательность словарных символов в корпусе. Так продолжается до тех пор, пока не будут выполнены слияния, создающие по крайней мере n новых токенов. В итоге, в окончательный словарь сходят все возможные частотные сочетания символов.

Далее применяется жадный парсер токенов, а словарь, полученный на основе корпуса, фактически становится набором правил, согласно которым парсер должен последовательно пройти по тексту, разделяя его и склеивая в словарные символы.

Таким образом, будучи примененным на больших объёмах данных BPE оставит словарные слова в их формах и только несловарная лексика будет разбита на составные части (как бы морфемы, но не совсем).